# Import everything

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

from torchsummary import summary
import torchvision
import torchvision.transforms as transforms

import os
import argparse

from tqdm.auto import tqdm, trange

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Download and Prepare Data

In [3]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


# Define Network

In [23]:
from model import MicroResNet, BasicBlock, ResNet

In [25]:
# def myNet():
#     # Trainable params: 1,235,274
#     # 50-epoch
#     # Train Acc = 88.6%, Test Acc = 86.4%
#     # 100-epoch
#     # Train Acc = 94.3%, Test Acc = 89.8%
#     # 200-epoch
#     # Train Acc = 99.9%, Test Acc = 93%
#     return MicroResNet(BasicBlock, [1, 1, 1])

In [26]:
# def myNet():
#     # Trainable params: 2,785,354
#     # 50-epoch
#     # Train Acc = 90.1%, Test Acc = 85.8%
#     # 100-epoch
#     # Train Acc = 93.8%, Test Acc = 88.5%
#     # 200-epoch
#     # Train Acc = 100%, Test Acc = 95.1%
#     return MicroResNet(BasicBlock, [2, 2, 2])

In [27]:
# def myNet():
#     # Trainable params: 4,903,242
#     # 50-epoch
#     # Train Acc = 89.3%, Test Acc = 83.8%
#     # 100-epoch
#     # Train Acc = 91.7%, Test Acc = 88.5%
#     # 200-epoch
#     # Train Acc = 100%, Test Acc = 94.4%
#     return ResNet(BasicBlock, [1, 1, 1, 1])

In [28]:
net = myNet()

In [29]:
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

In [30]:
net

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
    )
    (layer2): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1),

In [31]:
params = summary(net)

Layer (type:depth-idx)                   Param #
├─ResNet: 1-1                            --
|    └─Conv2d: 2-1                       1,728
|    └─BatchNorm2d: 2-2                  128
|    └─Sequential: 2-3                   --
|    |    └─BasicBlock: 3-1              73,984
|    └─Sequential: 2-4                   --
|    |    └─BasicBlock: 3-2              230,144
|    └─Sequential: 2-5                   --
|    |    └─BasicBlock: 3-3              919,040
|    └─Sequential: 2-6                   --
|    |    └─BasicBlock: 3-4              3,673,088
|    └─Linear: 2-7                       5,130
Total params: 4,903,242
Trainable params: 4,903,242
Non-trainable params: 0


In [32]:
print(f"Total Parameters: {params.total_params}, Trainable Parameter: {params.trainable_params}")
print(f"Satisfies the assignment criteria: {params.total_params <= 5000000}")

Total Parameters: 4903242, Trainable Parameter: 4903242
Satisfies the assignment criteria: True


# Hyperparameters

In [33]:
lr = 0.1
total_epochs = 200

# Helper Functions

In [34]:
best_acc = 0

In [35]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [36]:
def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    
    pbar = tqdm(total=len(trainloader), position=1)
    pbar.set_description('Train Epoch %i' % epoch)
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        loss = train_loss/(batch_idx+1)
        acc = 100.*correct/total
        
        pbar.set_postfix(loss=loss, acc=acc)
        pbar.update()

    pbar.close()

In [37]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        
        pbar = tqdm(total=len(testloader), position=1)
        pbar.set_description('Test Epoch %i' % epoch)
        
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            loss = test_loss/(batch_idx+1)
            acc = 100.*correct/total

            pbar.set_postfix(loss=loss, acc=acc)
            pbar.update()
        
        pbar.close()

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

# Train Network

In [38]:
start_epoch = 0

In [39]:
for epoch in trange(start_epoch, total_epochs, desc='Epoch'):
    train(epoch)
    test(epoch)
    scheduler.step()

Epoch:   0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

# Results